# Movie Analysis Dashboard

## Tasks:

## Data Preparation:
1. Reading Data: The code reads a movie dataset from a CSV file using Pandas.
2. Data Cleaning: Various data cleaning steps are performed, such as handling missing values and creating new columns like "boi" (Box Office Impact), "profit", etc.
3. Geospatial Data: Geospatial data for world countries is loaded using GeoPandas to map movies to their respective countries and continents.

## Data Visualization:
1. Budget vs. Revenue Scatter Plot: Visualizes the relationship between movie budget and revenue, with each point colored by its average rating. Brushing and linking are enabled to interactively explore the data.
2. Number of Movies Released per Year by Genre (Stacked Bar Chart): Shows the count of movies released per year, grouped by the top production countries. Allows selection of years using a point selection tool.
3. Pie Chart: Proportion of Movies by BOI (Based on Count): Represents the proportion of movies based on their Box Office Impact (BOI) using count aggregation.
4. Pie Chart: Proportion of Movies by BOI (Based on Budget): Illustrates the proportion of movies based on their BOI using budget aggregation.
5. Movie Lifespan by Genre (Box Plot): Displays the distribution of movie runtimes across different genres.

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd
import numpy as np
import timeit
from IPython.display import Image, display, HTML

import warnings
warnings.filterwarnings("ignore")

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

# 1. Dataset

## 1.1. Load CSV 

In [2]:
## Read csv
movies_df = pd.read_csv("data/movie_dataset.csv", parse_dates=['release_date'])

# show data
len(movies_df)

8747

In [3]:
movies_df = movies_df.sample(n=1000,random_state=40)

## 1.2. Load World Map Data

In [4]:
## Retrieve choropleth map metadata
local_map = "data/ne_50m_admin_0_countries.zip"
gdf_ne = gpd.read_file(local_map) 

# lower case all columns names
gdf_ne.columns = map(str.lower, gdf_ne.columns)

# Selects name, iso_a2, continent, pop_st and geometry column
gdf_ne = gdf_ne[["name", "iso_a2", "continent", "pop_est", 'geometry']]

# Map missing country code
gdf_ne.loc[gdf_ne["name"] == "Norway", 'iso_a2'] = "NO"
gdf_ne.loc[gdf_ne["name"] == "France", 'iso_a2'] = "FR"
gdf_ne.loc[gdf_ne["name"] == "Taiwan", 'iso_a2'] = "TW"

# Retrieve all of continents except for Antartica
gdf_ne = gdf_ne.query("continent in ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']")

## Generate iso_a2 -> continent map
iso_continent_df = pd.DataFrame(gdf_ne[["iso_a2", "continent", "name"]])
country_metadata_map = iso_continent_df.set_index('iso_a2').T.to_dict('index')
country_continent_map = country_metadata_map['continent']

In [5]:
## Add continents column to movies dataset
movies_df['continents'] = movies_df['iso_countries']
movies_df['continents'] = movies_df['continents'].replace(pd.Series(country_continent_map).astype(str), regex=True)
def remove_dupes(i1):  
    return ','.join(list(set(i1.split(", "))))

movies_df['continents'] = movies_df['continents'].apply(remove_dupes)

## 1.4. Create Genre DF

In [6]:
## split genres column
df_genre_split = movies_df.assign(genre=movies_df['genres'].str.split(', ')).explode('genre')


In [7]:
##Select first genre in genres column as main genre
movies_df["genre"] = movies_df["genres"].str.extract(r'([^,]+)')

# 2. Visualisation System

## 2.1. Define System Settings

In [8]:
SYSTEM_HEIGHT=750
SYSTEM_WIDTH=800

## 2.2. Data Adjustment

In [9]:
movies_data = movies_df

In [10]:
# Handle missing values in the 'budget' column by filling them with 'revenue' values
movies_data['budget'] = movies_data['budget'].fillna(movies_data['revenue'])
movies_data['budget'] = movies_data['budget'].mask(movies_data['budget'] == 0, movies_data['revenue'])

# Define a function to calculate Box Office Impact (BOI) based on revenue and budget
def calculate_boi(row):
    if row['revenue'] < row['budget']:
        return 'Flop'
    elif row['revenue'] >= 1.5 * row['budget'] and row['revenue'] < 2 * row['budget']:
        return 'Hit'
    elif row['revenue'] >= 2 * row['budget']:
        return 'Superhit'
    else:
        return 'Average'

# Apply the function to create a new column 'boi'
movies_data['boi'] = movies_data.apply(calculate_boi, axis=1)
movies_data['boi'] = movies_data['boi'].fillna('Average')

# Calculate profit by subtracting budget from revenue
movies_data['profit'] = movies_data['revenue'] - movies_data['budget']

# Abbr budget and profit by 1 million
million = 1e6
movies_data['budget_abbr']=round(movies_data['budget']/million, 3)
movies_data['revenue_abbr']=round(movies_data['revenue']/million, 3)

# Extracting year from release date
movies_data['release_year'] = movies_data['release_date'].dt.year

# Extract the first country's continent from the 'continents' column
movies_data['continent'] = movies_data["continents"].str.extract(r'([^,]+)') 
movies_data['production_country'] = movies_data["production_countries"].str.extract(r'([^,]+)')

# Explode the 'genres' column to analyze movie lifespan by genre
genres_df = movies_data.assign(genres=movies_data['genres'].str.split(', ')).explode('genres')

# movies_data.head(2)

## 2.3. Views

In [79]:
## START - BRUSH #######################################################################################################################
# Brushing & Linking
## View 1
budget_rev_brush = alt.selection_interval()

## View 2
year_selection = alt.selection_point(fields=['release_year'])


## View 4
genre_selection = alt.selection_point(fields=['genre'])

## END - BRUSH #########################################################################################################################


## START - VIEW 1 - #######################################################################################################################################
# 1. Budget vs. Revenue: Impact of Movie Ratings
revenue_min = movies_data["vote_average"].mean() - 2.5
revenue_max = movies_data["vote_average"].max()
revenue_color_scale = alt.Scale(
    scheme="yelloworangered",
    domain=[revenue_min, revenue_max]
)

base_chart = (
    alt.Chart(movies_data)
    .mark_square()
    .encode(
        x=alt.X(
            "budget:Q",
            title="Budget ($)"
        ),
        y=alt.Y(
            "revenue:Q",
            title="Revenue ($)"
        ),
        color=alt.Color(
            "vote_average:Q", 
            scale=revenue_color_scale,  
            title="Rating"
        ), 
        tooltip=[
            alt.Tooltip("title:N"),
            alt.Tooltip("budget:Q", title="Budget ($)"),  
            alt.Tooltip("revenue:Q", title="Revenue ($)"),
            alt.Tooltip("vote_average:Q", title="Rating"),
            alt.Tooltip("genre:N", title="Genre")
        ],
        opacity=alt.condition(budget_rev_brush, alt.OpacityValue(1), alt.OpacityValue(0.3))
    )
    .transform_calculate(release_year=alt.expr.year(alt.datum.release_date))
    .transform_filter(year_selection)
    .transform_filter(genre_selection)
    .add_params(budget_rev_brush)
    .properties(
        width=SYSTEM_WIDTH/2, 
        height=400, 
        title="Budget vs. Revenue: Impact of Movie Ratings",
    )
)

# Axis selections (to drive the dynamic behavior)
x_select = alt.selection_interval(encodings=["x"], name="x_axis") 
y_select = alt.selection_interval(encodings=["y"], name="y_axis") 

# Chart showing only selected points when brushed
selected_points = (
    alt.Chart(movies_data)
    .mark_square(size=50)
    .encode(
        x=alt.X(
            "budget:Q", 
            title="Budget ($)", 
            scale=alt.Scale(domain=x_select)  # Connect to x_select
        ),
        y=alt.Y(
            "revenue:Q", 
            title="Revenue ($)", 
            scale=alt.Scale(domain=y_select)  # Connect to y_select
        ),
        color=alt.Color(
            "vote_average:Q", 
            scale=revenue_color_scale,  
            title="Rating"
        ),
        tooltip=[
            alt.Tooltip("title:N"),
            alt.Tooltip("budget:Q", title="Budget ($)"),  
            alt.Tooltip("revenue:Q", title="Revenue ($)"),
            alt.Tooltip("vote_average:Q", title="Rating"),
            alt.Tooltip("boi:N", title="Verdict")
        ]
    )
    .transform_calculate(release_year=alt.expr.year(alt.datum.release_date))
    .transform_filter(year_selection)
    .transform_filter(genre_selection)
    .transform_filter(budget_rev_brush)  # Respond to both selections
    # .transform_filter(click)
    # .add_params(x_select, y_select) # Add the axis selections
).interactive()
## END - VIEW 1 - #######################################################################################################################################




## START - VIEW 2 - #######################################################################################################################################
# 2. Stacked Bar Chart: Number of Movies Released per Year by Country
stack_bar_chart = alt.Chart(movies_data).mark_bar().encode(
    x=alt.X('release_year:O', title="Year Released", axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('counts:Q', stack='zero', title="Count of movies released"),
    color=alt.Color('top_country:N',scale=alt.Scale(scheme='set3'), title="Production Countries"), 
    tooltip=[
        alt.Tooltip('release_year:O', title='Year Released'),
        alt.Tooltip('top_country:N', title='Production Country'),
        alt.Tooltip('counts:Q', title='Count of Movies') 
    ],
    opacity=alt.condition(year_selection, alt.OpacityValue(1), alt.OpacityValue(0.3)),
    # order=alt.Order('counts:O', sort='ascending')
).transform_calculate(
    release_year=alt.expr.year(alt.datum.release_date)
).transform_filter(
    genre_selection
).transform_filter(
    budget_rev_brush
).transform_aggregate(
    counts='count(id)',
    groupby=["release_year", "production_country"]
).transform_window(
    rank='row_number(counts)',
    sort=[alt.SortField('counts', order='descending')],
    groupby=['release_year']
).transform_calculate(
    top_country = "datum.rank <= 6 ? datum.production_country : '_Others'"
).transform_aggregate(
    counts='sum(counts)',
    groupby=["release_year", "top_country"]
).add_params(
    year_selection
).properties(
    width=SYSTEM_WIDTH,
    height=450,
    title='Number of Movies Released per Year by Country'
)
## END - VIEW 2 - #######################################################################################################################################





## START - VIEW 3 - #######################################################################################################################################
# 4. Pie Chart: Proportion of Movies by BOI (Based on Count)
pie_chart = alt.Chart(movies_data).mark_arc().encode(
    theta='count:Q',
    color=alt.Color('boi:N', scale=alt.Scale(scheme='category20'), title="Verdict"),  
    tooltip=[
        alt.Tooltip('boi:N', title='Verdict'),
        alt.Tooltip('count:Q', title='Count of Movies Released'),
        alt.Tooltip('percentage:Q', title='Percentage of Total Movies', format='.2%')
    ]
).transform_calculate(
    release_year=alt.expr.year(alt.datum.release_date)
).transform_filter(
    genre_selection 
).transform_filter(
    year_selection
).transform_filter(
    budget_rev_brush
).transform_aggregate(
    count='count():Q',
    groupby=['boi']
).transform_joinaggregate(
    total_count='sum(count)',
).transform_calculate(
    percentage="datum.count/datum.total_count"
).properties(
    width=SYSTEM_WIDTH/2,
    height=270,
    title="Proportion of Movies by BOI (Based on Count)"
)

# 5. Pie Chart: Proportion of Movies by BOI (Based on Budget)
pie_chart2 = alt.Chart(movies_data).mark_arc().encode(
    theta=alt.Theta('sum_budget:Q', title='Budget Proportion'),
    color=alt.Color('boi:N', scale=alt.Scale(scheme='category20'), title="Verdict"),  
    tooltip=[
        alt.Tooltip('boi:N', title='Verdict'),
        alt.Tooltip('sum_budget:Q', title='Total Budget'), 
        alt.Tooltip('percentage:Q', title='Percentage of Movies', format='.2%')
    ]
).transform_calculate(
    release_year=alt.expr.year(alt.datum.release_date)
).transform_filter(
    year_selection
).transform_filter(
    genre_selection 
).transform_filter(
    budget_rev_brush
).transform_aggregate(
    sum_budget='sum(budget):Q',
    groupby=['boi']
).transform_joinaggregate(
    total_budget='sum(sum_budget)'
).transform_calculate(
    percentage="datum.sum_budget / datum.total_budget"
).properties(
    width=SYSTEM_WIDTH/2,
    height=270,
    title="Proportion of Movies by BOI (Based on Budget)"
)

# Combine base chart and selected points
combined_charts = (pie_chart | pie_chart2)
## END - VIEW 3 - #######################################################################################################################################




## START - VIEW 4 - #######################################################################################################################################
# 4. Box Plot: Movie Lifespan by Genre

genre_color = alt.condition(genre_selection,
                      alt.Color('genre:N', legend=None),
                      alt.value('lightgray')) ##use this coloring???
genre_list = pd.DataFrame({'genre': movies_data['genre'].unique()}) 
genre_legend = alt.Chart(genre_list).mark_point(size=200, filled=True).encode(
    y=alt.Y('genre:N', axis=alt.Axis(orient='right')),
    color=genre_color
).add_params(genre_selection)



lifespan_chart = alt.Chart(movies_data).mark_boxplot().encode(
    x= alt.X('genre:N', title = "Genre", axis=alt.Axis(labelAngle=-45)),
    y= alt.X('runtime:Q', title = "Runtime"),
    color=alt.Color('genre:N',scale=alt.Scale(scheme='category20'), legend=None),
    tooltip=[
        alt.Tooltip('title', title='Title'),
        alt.Tooltip('runtime', title='Runtime'),
        ]
).transform_calculate(
    release_year=alt.expr.year(alt.datum.release_date)
).transform_filter(
    genre_selection 
).transform_filter(
    year_selection
).transform_filter(
    budget_rev_brush
).properties(
    width = SYSTEM_WIDTH,
    height = 500,
    title = 'Movie Lifespan by Genre'
)

## END - VIEW 4 - #######################################################################################################################################





# Concatenate charts vertically
rating_chart2 = alt.vconcat(
    (base_chart | selected_points), 
    stack_bar_chart,
    combined_charts,
    (lifespan_chart | genre_legend),
)

# Display the final chart
rating_chart2



alt.VConcatChart(...)